In [1]:
#### Now since this is a new file we need to import other modules (YAY!) including our R' emulator
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

### and let's not forget the usual gang:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
import os 

In [2]:
#### Before we get started in an R' environment. We need to talk about something important.
#### Rentrez does not use SNP ID to search. It uses UID. As in, they don't use 'rs2000' 
#### but rather '2000'. So we need to use our previous output and split the function.
Output_GeneSNP_file = pd.read_csv('Resources/OutputFromExtractionCode/Editable_Gene_And_dbSNP_ID_Merged.csv')
Output_GeneSNP_file.head(10)

output_path = 'Resources/OutputFromExtractionCode'

In [3]:
### Now we split it. 
Split_dbSNP_ID = Output_GeneSNP_file
Split_dbSNP_ID[['rs', 'UID']] = Output_GeneSNP_file['SNP ID'].str.split('rs', expand = True)
Split_dbSNP_ID.head(10)

#And delete the rs column:
UID_SNP = Split_dbSNP_ID.drop(columns=['rs'])
UID_SNP.head(10)

## Now we have the UID

,GENE,SNP ID,UID
0,BOLL,rs700651,700651
1,[U]EDNRA,rs6841581,6841581
2,PTCH1,rs1105980,1105980
3,CCDC3,rs12412014,12412014
4,LINC00457,rs1536847,1536847
5,C5orf60,rs62405726,62405726
6,METTL4,rs549315,549315
7,RGPD4,rs700855,700855
8,RGPD4,rs328025,328025
9,MALL,rs117802391,117802391


In [4]:
### Let's import RENTREZ to our python environment
rentrez = importr('rentrez')

### You'll need to provide an email address to use Entrez tools. BTW Hot tip: 
### If you have multiple lines, using triple quotes is cleaner and easier to read:

robjects.r('''
library(rentrez)
entrez_email <- "kenji.ponsm@gmail.com"
''')

In [5]:
### What can we look for with entrez
snp_database = robjects.r('entrez_db_searchable("snp")')
print(snp_database)

Searchable fields for database 'snp'
  ALL 	 All terms from all searchable fields 
  UID 	 Unique number assigned to publication 
  FILT 	 Limits the records 
  RS 	 Clustered SNP ID (rs) 
  CHR 	 chromosomes 
  GENE 	 locus link symbol 
  HAN 	 Submitter Handle 
  ACCN 	 nucleotide accessions 
  GENE_ID 	 Gene ID 
  FXN 	 dbSNP Functional consequence class 
  GTYP 	 Genotype info 
  SS 	 Submitter ID 
  VARI 	 Allele 
  SCLS 	 SNP class 
  CPOS 	 Chromosome base position 
  WORD 	 Free text associated with record 
  SIDX 	 SNP Index 
  CLIN 	 Variations with clinical effects or significances 
  GMAF 	 Minor Allele Frequency derived from global population (ie. 1000G) 
  VALI 	 Validation status 
  CPOS_GRCH37 	 Chromosome base position on previous assembly version 
  ORGN 	 Organism 
  ALFA_EUR 	 ALFA European Minor Allele Frequency 
  ALFA_AFR 	 ALFA African population Minor Allele Frequency 
  ALFA_ASN 	 ALFA Asian population Minor Allele Frequency 
  ALFA_LAC 	 ALFA Latin American 1

In [6]:
## Let's give it a test shall we? Let's say I want the information of rs429358 (GENE: APOE)
print(UID_SNP.iloc[0,2])
test_snp = UID_SNP.iloc[0,2]
search_test = rentrez.entrez_summary(db='snp', id=test_snp)
search_test_dict = {key: str(search_test.rx2(key) [0]) for key in search_test.names}

search_test_df = pd.DataFrame([search_test_dict])

search_test_df.head(10)



700651


,uid,snp_id,allele_origin,global_mafs,global_population,global_samplesize,suspected,clinical_significance,genes,acc,...,allele,snp_class,chrpos,chrpos_prev_assm,text,snp_id_sort,clinical_sort,cited_sort,chrpos_sort,merged_sort
0,700651,700651,,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",,,,,"[1] ""BOLL""\n",NC_000002.12,...,R,snv,2:197766990,2:198631714,,0000700651,0,,0197766990,0


In [7]:
print(search_test.names)


 [1] "uid"                   "snp_id"                "allele_origin"        
 [4] "global_mafs"           "global_population"     "global_samplesize"    
 [7] "suspected"             "clinical_significance" "genes"                
[10] "acc"                   "chr"                   "handle"               
[13] "spdi"                  "fxn_class"             "validated"            
[16] "docsum"                "tax_id"                "orig_build"           
[19] "upd_build"             "createdate"            "updatedate"           
[22] "ss"                    "allele"                "snp_class"            
[25] "chrpos"                "chrpos_prev_assm"      "text"                 
[28] "snp_id_sort"           "clinical_sort"         "cited_sort"           
[31] "chrpos_sort"           "merged_sort"          



In [8]:
### And let's filter for the actual values we want to extract: 
desired_dbSNP_values = (['genes', 'uid', 'snp_class', 
                         'fxn_class', 'chr', 'chrpos', 'global_mafs', 
                         'allele','clinical_significance', 'acc'] 
                         )
filtered_search_test_dict = {key: str(search_test.rx2(key)[0]) for key in search_test.names if key in desired_dbSNP_values}
filtered_search_test = pd.DataFrame([filtered_search_test_dict])
filtered_search_test

,uid,global_mafs,clinical_significance,genes,acc,chr,fxn_class,allele,snp_class,chrpos
0,700651,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",,"[1] ""BOLL""\n",NC_000002.12,2,intron_variant,R,snv,2:197766990


In [9]:
filtered_and_ordered_search_test = filtered_search_test[[col for col in desired_dbSNP_values if col in filtered_search_test.columns]]
filtered_and_ordered_search_test


,genes,uid,snp_class,fxn_class,chr,chrpos,global_mafs,allele,clinical_significance,acc
0,"[1] ""BOLL""\n",700651,snv,intron_variant,2,2:197766990,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",R,,NC_000002.12


In [10]:
all_snp_data = []

# Open a text file in write mode to log skipped rows
with open("skipped_rows_log.txt", "w") as log_file:
    log_file.write("Details of rows skipped due to None UID:\n\n")

    for index, row in UID_SNP.iterrows():
        test_snp = row['UID']
        
        # Check if the UID is not None
        if test_snp is None:
            # Print to console and write to log file
            log_message = f"Skipping row {index} because UID is None. Row details:\n{row}\n\n"
            print(log_message)
            log_file.write(log_message)
            continue  # Skip this iteration if UID is None

        try: 
            # Retrieve SNP data from the SNP database
            search_test = rentrez.entrez_summary(db='snp', id=test_snp)
            
            # Filter and safely retrieve desired fields
            filtered_search_test_dict = {
                key: str(search_test.rx2(key)[0]) if len(search_test.rx2(key)) > 0 else None
                for key in search_test.names if key in desired_dbSNP_values
            }
            
            # Order the data to match the desired column order
            ordered_data = {key: filtered_search_test_dict.get(key, None) for key in desired_dbSNP_values}
            
            # Append ordered data to the results list
            all_snp_data.append(ordered_data)

        except Exception as e:
            error_message = f"Error retrieving data for UID {test_snp}: {e}\n"
            print(error_message)
            log_file.write(error_message)

# Convert results to DataFrame
final_df = pd.DataFrame(all_snp_data)

# Display the first 50 rows
final_df.head(50)


,genes,uid,snp_class,fxn_class,chr,chrpos,global_mafs,allele,clinical_significance,acc
0,"[1] ""BOLL""\n",700651,snv,intron_variant,2,2:197766990,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",R,,NC_000002.12
1,"[1] ""EDNRA""\n",6841581,snv,"2KB_upstream_variant,upstream_transcript_variant",4,4:147480038,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",N,,NC_000004.12
2,None,1105980,snv,,9,9:95351353,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",S,,NC_000009.12
3,None,12412014,snv,,10,10:12869725,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",B,,NC_000010.11
4,"[1] ""LINC00457"" ""LINC02343""\n",1536847,snv,intron_variant,13,13:34532838,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",D,,NC_000013.11
5,"[1] ""C5orf60""\n",62405726,snv,intron_variant,5,5:179642467,"[1] ""1000Genomes"" ""ExAC"" ""GnomAD"" ...",N,,NC_000005.10
6,None,549315,snv,,18,18:2183055,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",V,,NC_000018.10
7,"[1] ""GACAT1""\n",700855,snv,"upstream_transcript_variant,2KB_upstream_variant",2,2:107752238,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",Y,,NC_000002.12
8,None,328025,snv,,2,2:107738589,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",D,,NC_000002.12
9,"[1] ""MALL""\n",117802391,snv,intron_variant,2,2:110104507,"[1] ""1000Genomes"" ""ALSPAC"" ""GnomAD...",Y,,NC_000002.12


In [11]:
#### Now let's make a copy of our RAW results:
final_df.to_csv(os.path.join(output_path, 'compiled_SNPs_RENTREZ_RAW.csv'), index=False)


In [12]:
#### And let's make it beautiful by rearranging and sorting each column:
#### First, let's delete all variants that are not SNVs:
filtered_final_df = final_df.loc[final_df['snp_class'] == 'snv'] 
filtered_final_df

,genes,uid,snp_class,fxn_class,chr,chrpos,global_mafs,allele,clinical_significance,acc
0,"[1] ""BOLL""\n",700651,snv,intron_variant,2,2:197766990,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",R,,NC_000002.12
1,"[1] ""EDNRA""\n",6841581,snv,"2KB_upstream_variant,upstream_transcript_variant",4,4:147480038,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",N,,NC_000004.12
2,None,1105980,snv,,9,9:95351353,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",S,,NC_000009.12
3,None,12412014,snv,,10,10:12869725,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",B,,NC_000010.11
4,"[1] ""LINC00457"" ""LINC02343""\n",1536847,snv,intron_variant,13,13:34532838,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",D,,NC_000013.11
...,...,...,...,...,...,...,...,...,...,...
284,"[1] ""THSD1""\n",186046951,snv,"coding_sequence_variant,missense_variant",13,13:52397661,"[1] ""1000Genomes"" ""ExAC"" ""GnomAD"" ...",S,,NC_000013.11
285,"[1] ""TMEM132B""\n",61940807,snv,"coding_sequence_variant,genic_downstream_trans...",12,12:125654210,"[1] ""1000Genomes"" ""ALSPAC"" ""Estonian...",M,,NC_000012.12
286,"[1] ""LOX"" ""SRFBP1""\n",10519694,snv,"genic_downstream_transcript_variant,intron_var...",5,5:122071524,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",Y,,NC_000005.10
287,"[1] ""LOXL2""\n",1010156,snv,"coding_sequence_variant,synonymous_variant",8,8:23333428,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",H,,NC_000008.11


In [13]:
### Now let's clean up la mugre of each cell in the genes column

# Define a function to clean up each cell
def clean_cell(value):
    if isinstance(value, str):
        # Remove `[1]`, quotation marks, and newline characters
        cleaned_value = re.sub(r'^\[\d+\]\s*"', '', value).replace('"', '').strip()
        return cleaned_value
    return value

# Apply the cleaning function to the 'genes' column using .loc
filtered_final_df.loc[:, 'genes'] = filtered_final_df['genes'].apply(clean_cell)

filtered_final_df.head(200)

,genes,uid,snp_class,fxn_class,chr,chrpos,global_mafs,allele,clinical_significance,acc
0,BOLL,700651,snv,intron_variant,2,2:197766990,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",R,,NC_000002.12
1,EDNRA,6841581,snv,"2KB_upstream_variant,upstream_transcript_variant",4,4:147480038,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",N,,NC_000004.12
2,None,1105980,snv,,9,9:95351353,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",S,,NC_000009.12
3,None,12412014,snv,,10,10:12869725,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",B,,NC_000010.11
4,LINC00457 LINC02343,1536847,snv,intron_variant,13,13:34532838,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",D,,NC_000013.11
...,...,...,...,...,...,...,...,...,...,...
195,LOC105376234,4979583,snv,"genic_downstream_transcript_variant,intron_var...",9,9:115809149,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",H,,NC_000009.12
196,OLFML2A,79134766,snv,"upstream_transcript_variant,missense_variant,c...",9,9:124799444,"[1] ""1000Genomes"" ""Estonian"" ""GnomAD...",D,,NC_000009.12
197,LOC105369370,76855873,snv,intron_variant,11,11:69339687,"[1] ""1000Genomes"" ""ALSPAC"" ""GnomAD"" ...",Y,,NC_000011.10
198,ARHGAP32,371331393,snv,"stop_gained,genic_downstream_transcript_varian...",11,11:128969377,"[1] ""GnomAD"" ""GoESP"" ""PAGE_STUDY"" ""TO...",R,,NC_000011.10


In [14]:
# Create a copy to avoid the SettingWithCopyWarning
filtered_final_df = filtered_final_df.copy()

# Rename columns
filtered_final_df.rename(columns={'genes': 'Gene ID (GENE_ID)', 'uid': 'dbSNP ID (RS)',
                                  'snp_class': 'SNP Class (SCLS)', 'fxn_class': 'Functional consequence class (FXN)', 
                                  'chr': 'Chromosome', 'chrpos': 'Chromosome base position (CPOS)', 
                                  'global_mafs': 'Minor Allele Frequency from global population (GMAF)',
                                  'clinical_significance': 'Associated clinical phenotypes (CLIN)',
                                  'acc': 'Nucleotide Accessions (ACCN)', 'allele': 'Allele'                                 
                                 }, inplace=True)

# IUPAC mapping dictionary for ambiguity codes
iupac_mapping = {
    "R": "A/G", "Y": "C/T", "S": "G/C", "W": "A/T",
    "K": "G/T", "M": "A/C", "B": "C/G/T", "D": "A/G/T",
    "H": "A/C/T", "V": "A/C/G", "N": "A/T/C/G"
}

# Define a function to translate IUPAC codes in the 'allele' column
def translate_allele(allele):
    if isinstance(allele, str):  # Check if the value is a string
        # Replace each IUPAC code with its corresponding alleles
        return "/".join([iupac_mapping.get(a, a) for a in allele])
    return allele

# Assume filtered_final_df is your DataFrame and 'allele' is the column with IUPAC codes
# Apply the translation function to the 'allele' column
filtered_final_df['Allele'] = filtered_final_df['Allele'].apply(translate_allele)

# Display the updated DataFrame
filtered_final_df.head(5)


,Gene ID (GENE_ID),dbSNP ID (RS),SNP Class (SCLS),Functional consequence class (FXN),Chromosome,Chromosome base position (CPOS),Minor Allele Frequency from global population (GMAF),Allele,Associated clinical phenotypes (CLIN),Nucleotide Accessions (ACCN)
0,BOLL,700651,snv,intron_variant,2,2:197766990,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",A/G,,NC_000002.12
1,EDNRA,6841581,snv,"2KB_upstream_variant,upstream_transcript_variant",4,4:147480038,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",A/T/C/G,,NC_000004.12
2,None,1105980,snv,,9,9:95351353,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",G/C,,NC_000009.12
3,None,12412014,snv,,10,10:12869725,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",C/G/T,,NC_000010.11
4,LINC00457 LINC02343,1536847,snv,intron_variant,13,13:34532838,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",A/G/T,,NC_000013.11


In [15]:
import requests
import pandas as pd

# Function to get Gene ID and Gene Type from Ensembl REST API
def get_gene_info_via_ensembl(gene_symbol, species="homo_sapiens"):
    # Construct the URL for Ensembl's REST API
    url = f"https://rest.ensembl.org/lookup/symbol/{species}/{gene_symbol}"
    
    # Set the headers to request JSON format
    headers = {"Content-Type": "application/json"}
    
    try:
        # Send the GET request to Ensembl's API
        response = requests.get(url, headers=headers)
        
        # Check if the response is successful
        if response.status_code == 200:
            data = response.json()
            
            # Extract Gene ID and Gene Type
            gene_id = data.get("id", "Gene ID not available")
            gene_type = data.get("biotype", "Gene type not available")
            
            # Check if Gene Type is None and replace it
            if gene_type is None:
                gene_type = "No gene reported"
            
            print(f"Gene Symbol: {gene_symbol}, Ensembl Gene ID: {gene_id}, Gene Type: {gene_type}")
            return gene_id, gene_type
        elif response.status_code == 400:
            print(f"No gene found for symbol '{gene_symbol}' in Ensembl.")
            return "No gene found", "Intergenic (Review)"
        else:
            print(f"Failed to retrieve data for {gene_symbol}. HTTP status: {response.status_code}")
            return None, "Error retrieving data"
    except Exception as e:
        print(f"An error occurred for gene {gene_symbol}: {e}")
        return None, "Error retrieving data"

# Assume filtered_final_df is your existing DataFrame with the gene symbols in a column named 'Gene Symbol'
# Replace 'Gene Symbol' with the name of your actual column containing the gene symbols
filtered_final_df[['Ensembl Gene ID', 'Gene Type']] = filtered_final_df['Gene ID (GENE_ID)'].apply(
    lambda gene: pd.Series(get_gene_info_via_ensembl(gene))
)


Gene Symbol: BOLL, Ensembl Gene ID: ENSG00000152430, Gene Type: protein_coding
Gene Symbol: EDNRA, Ensembl Gene ID: ENSG00000151617, Gene Type: protein_coding
No gene found for symbol 'None' in Ensembl.
No gene found for symbol 'None' in Ensembl.
No gene found for symbol 'LINC00457 LINC02343' in Ensembl.
Gene Symbol: C5orf60, Ensembl Gene ID: ENSG00000204661, Gene Type: protein_coding
No gene found for symbol 'None' in Ensembl.
Gene Symbol: GACAT1, Ensembl Gene ID: ENSG00000232991, Gene Type: lncRNA
No gene found for symbol 'None' in Ensembl.
Gene Symbol: MALL, Ensembl Gene ID: ENSG00000144063, Gene Type: protein_coding
No gene found for symbol 'LINC01978 LINC01979' in Ensembl.
Gene Symbol: RREB1, Ensembl Gene ID: ENSG00000124782, Gene Type: protein_coding
Gene Symbol: DST, Ensembl Gene ID: ENSG00000151914, Gene Type: protein_coding
No gene found for symbol 'None' in Ensembl.
Gene Symbol: FOXP1, Ensembl Gene ID: ENSG00000114861, Gene Type: protein_coding
Gene Symbol: LINC01344, Ensembl

In [16]:
####Now let's output this table.
filtered_final_df.to_csv(os.path.join(output_path, 'RENTREZ_NCBI_Compiled.csv'), index=False)


In [17]:
### Again unique genes
Unique_Genes = filtered_final_df['Gene ID (GENE_ID)'].value_counts().reset_index()
Unique_Genes.columns = ['Gene ID (GENE_ID)', 'Reported SNPs']
Unique_Genes.head(20)

,Gene ID (GENE_ID),Reported SNPs
0,RNF213,13
1,VCAN,11
2,CDKN2B-AS1,11
3,PCNT,9
4,MMP2,9
5,MMP24 MMP24-AS1-EDEM2,6
6,JDP2,6
7,MMP26,6
8,LIMK1,6
9,RNF213 RNF213-AS1,5


In [18]:
# Assuming filtered_final_df is your DataFrame
# Filter rows where Gene Type is 'protein-coding'
protein_coding_df = filtered_final_df[filtered_final_df['Gene Type'] == 'protein_coding']

# Display the filtered DataFrame
protein_coding_df.head(10)


,Gene ID (GENE_ID),dbSNP ID (RS),SNP Class (SCLS),Functional consequence class (FXN),Chromosome,Chromosome base position (CPOS),Minor Allele Frequency from global population (GMAF),Allele,Associated clinical phenotypes (CLIN),Nucleotide Accessions (ACCN),Ensembl Gene ID,Gene Type
0,BOLL,700651,snv,intron_variant,2,2:197766990,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",A/G,,NC_000002.12,ENSG00000152430,protein_coding
1,EDNRA,6841581,snv,"2KB_upstream_variant,upstream_transcript_variant",4,4:147480038,"[1] ""1000Genomes"" ""ALSPAC"" ""Chilea...",A/T/C/G,,NC_000004.12,ENSG00000151617,protein_coding
5,C5orf60,62405726,snv,intron_variant,5,5:179642467,"[1] ""1000Genomes"" ""ExAC"" ""GnomAD"" ...",A/T/C/G,,NC_000005.10,ENSG00000204661,protein_coding
9,MALL,117802391,snv,intron_variant,2,2:110104507,"[1] ""1000Genomes"" ""ALSPAC"" ""GnomAD...",C/T,,NC_000002.12,ENSG00000144063,protein_coding
11,RREB1,9505086,snv,intron_variant,6,6:7231953,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",C/T,,NC_000006.12,ENSG00000124782,protein_coding
12,DST,117021265,snv,"intron_variant,genic_upstream_transcript_variant",6,6:56763223,"[1] ""1000Genomes"" ""GnomAD"" ""KOREAN"" ...",C/T,,NC_000006.12,ENSG00000151914,protein_coding
14,FOXP1,878118,snv,"genic_upstream_transcript_variant,genic_downst...",3,3:71197077,"[1] ""1000Genomes"" ""ALSPAC"" ""Daghes...",A/G/T,,NC_000003.12,ENSG00000114861,protein_coding
16,RBMS3,1979271,snv,"genic_upstream_transcript_variant,intron_variant",3,3:29565914,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",A/T/C/G,,NC_000003.12,ENSG00000144642,protein_coding
19,RUFY1,4075890,snv,"intron_variant,genic_upstream_transcript_variant",5,5:179570372,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",C/G/T,,NC_000005.10,ENSG00000176783,protein_coding
21,PLEKHA1,10510110,snv,"non_coding_transcript_variant,genic_downstream...",10,10:122432914,"[1] ""1000Genomes"" ""ALSPAC"" ""Estoni...",A/T/C/G,,NC_000010.11,ENSG00000107679,protein_coding


In [19]:

# Assuming filtered_final_df is your DataFrame with gene symbols and gene types

# Filter rows with 'protein-coding' in the Gene Type column
protein_coding_df = filtered_final_df[filtered_final_df['Gene Type'] == 'protein_coding']

# Extract the 'Gene ID (GENE_ID)' column as a list
protein_coding_genes = protein_coding_df['Gene ID (GENE_ID)'].tolist()

# Define the output path for the text file
output_path_txt = f'{output_path}/protein_coding_genes.txt'  # You can set this to any desired path

# Save the list to a text file with each gene on a new line
with open(output_path_txt, "w") as file:
    for gene in protein_coding_genes:
        file.write(f"{gene}\n")

print(f"Protein-coding genes have been saved to '{output_path}'")

# Display the first few items in the list for verification
print(protein_coding_genes[:5])  # Shows the first 5 genes


Protein-coding genes have been saved to 'Resources/OutputFromExtractionCode'
['BOLL', 'EDNRA', 'C5orf60', 'MALL', 'RREB1']
